#ECE 225 Project Model Prediction

In [1]:
import numpy as np 
import pandas as pd

df = pd.read_csv('artist_data_combined.csv')

df.drop(['Unnamed: 0'], axis = 1, inplace=True)

In [3]:
print(df.shape)

#Data Cleaning 
Null_values = df.isnull().sum()
df.dropna(inplace = True)
df.drop_duplicates(inplace = True)
print(df.shape) #Looking at the change in shape, looks like there were some missing values and duplicates

(5318, 19)
(5318, 19)


In [4]:
def get_first_name(x):
    if type(x) is float:
        return ''
    return x.strip('][').split(', ')[0].replace("'",'')



df['First Genre'] = df['Main Genres'].apply(get_first_name)

In [6]:
df['First Genre']

1              Rock
2       Hip Hop/Rap
3       Hip Hop/Rap
4       Hip Hop/Rap
5       Hip Hop/Rap
           ...     
8560          Other
8562           Jazz
8564            Pop
8566           Jazz
8567            Pop
Name: First Genre, Length: 5318, dtype: object

In [7]:
genre_list = list(set(df['First Genre'].values.tolist()))

values = list(range(len(genre_list)))
key = genre_list
dictionary = dict(zip(key,values))
dictionary

{'Christian': 16,
 'Christmas': 7,
 'Classical': 10,
 'Country': 0,
 'Disco': 8,
 'Disney': 12,
 'Electronic': 1,
 'Hip Hop/Rap': 13,
 'Indie': 9,
 'Jazz': 3,
 'Latin': 14,
 'Metal': 15,
 'Other': 2,
 'Pop': 11,
 'Punk': 5,
 'R&B': 6,
 'Rock': 4}

In [8]:
y = df['First Genre'].apply(lambda x:dictionary[x])
y

1        4
2       13
3       13
4       13
5       13
        ..
8560     2
8562     3
8564    11
8566     3
8567    11
Name: First Genre, Length: 5318, dtype: int64

In [9]:
df

,mode,count,acousticness,Name,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,Genres,artist_id,Monthly Listeners,Main Genres,First Genre
1,1,122,0.173145,"""Weird Al"" Yankovic",0.662787,218948.196721,0.695393,0.000050,0.161102,-9.768705,0.084536,133.031180,0.751344,34.229508,9.0,"['comedy rock', 'comic', 'parody']",1bDWGdIC2hardyt55nlQgG,863725.0,['Rock'],Rock
2,1,15,0.544467,$NOT,0.789800,137910.466667,0.532933,0.023063,0.180300,-9.149267,0.293687,112.344800,0.480700,67.533333,1.0,"['florida rap', 'sad rap', 'underground hip ho...",5IbEL2xjRtKsunfmsahLuO,5193207.0,"['Hip Hop/Rap', 'Hip Hop/Rap']",Hip Hop/Rap
3,1,2,0.239000,$atori Zoom,0.883000,141519.000000,0.625000,0.000000,0.076500,-4.098000,0.245000,126.677000,0.871000,67.000000,6.0,"['dark trap', 'meme rap']",75RLThQWzIgIM7Jtp6KZic,374482.0,['Hip Hop/Rap'],Hip Hop/Rap
4,1,2,0.148100,$tupid Young,0.854000,190572.000000,0.683000,0.000002,0.188500,-6.997000,0.221000,100.724500,0.625500,57.500000,1.0,"['asian american hip hop', 'cali rap', 'west c...",0h0C8mMT21p0Ixq6qO1UMH,589040.0,"['Hip Hop/Rap', 'Hip Hop/Rap']",Hip Hop/Rap
5,1,125,0.141485,$uicideBoy$,0.749344,146386.392000,0.635552,0.045675,0.202253,-6.631304,0.156108,115.022024,0.287286,61.800000,1.0,"['dark trap', 'new orleans rap', 'underground ...",1VPmR4DJC1PlOtd0IADAO0,6591484.0,"['Hip Hop/Rap', 'Hip Hop/Rap']",Hip Hop/Rap
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8560,1,5,0.694800,John Kander,0.562200,264477.400000,0.405000,0.000000,0.562600,-11.610800,0.203620,109.875800,0.570800,20.800000,9.0,['hollywood'],7xSEQmlssbeoIUdQ0HFjUT,38884.0,['Other'],Other
8562,1,9,0.502584,John Klemmer,0.528222,267975.000000,0.358000,0.208587,0.281256,-13.960444,0.037011,99.546000,0.509222,29.555556,5.0,"['jazz funk', 'smooth jazz']",6HkwRgGXuqTufwpEh2dwGK,37780.0,['Jazz'],Jazz
8564,1,75,0.358409,John Legend,0.557213,244990.453333,0.594493,0.000026,0.210460,-7.136760,0.097996,116.262187,0.432968,57.733333,7.0,"['neo mellow', 'neo soul', 'pop', 'pop soul', ...",5y2Xq6xcjJb2jVM54GHK3t,26668894.0,"['Pop', 'R&B']",Pop
8566,1,34,0.959794,John Lewis,0.537618,227513.823529,0.107991,0.690242,0.146888,-20.320382,0.052250,102.432559,0.329441,10.588235,0.0,"['bebop', 'contemporary post-bop', 'cool jazz'...",7G7zZjjTwBZ9vzaoogk95s,33148.0,['Jazz'],Jazz


In [10]:
df.drop(['Name', 'Genres', 'artist_id', 'Main Genres', 'First Genre'], axis = 1, inplace=True)

#Train / Test Split

Before we start with our analysis, we have to split the data into Test/Train splits. It can be done in two ways. Either we can randomly sample our data like we have been doing in class or we can use stratified sampling to make sure that we get an even representation from all classes. 
It would be interesting to see how much of a different these two different sampling techniques would make. 

In [19]:
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, y, train_size=0.80, stratify=y)
print('Frequency of each class with just random sampling in Y Train is ', Counter(y_train))
print('Frequency of each class with just random sampling in Y Train is ', Counter(y_test))

Frequency of each class with just random sampling in Y Train is  Counter({11: 1293, 2: 1026, 4: 589, 13: 336, 3: 246, 1: 123, 10: 114, 14: 104, 6: 76, 0: 74, 15: 62, 5: 50, 8: 48, 9: 42, 16: 38, 12: 24, 7: 9})
Frequency of each class with just random sampling in Y Train is  Counter({11: 324, 2: 257, 4: 147, 13: 84, 3: 61, 1: 31, 10: 28, 14: 26, 6: 19, 0: 18, 15: 15, 5: 13, 8: 12, 9: 11, 16: 10, 12: 6, 7: 2})


In [13]:
X_train

,mode,count,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,Monthly Listeners
3123,1,56,0.857464,0.625339,158061.178571,0.265436,0.692468,0.117087,-15.056393,0.042191,103.980125,0.496054,24.428571,0.0,448911.0
6462,1,2,0.113000,0.728000,241533.000000,0.477000,0.005840,0.132000,-8.687000,0.172000,173.195000,0.731000,34.000000,7.0,10278.0
4365,0,2,0.004540,0.843000,237040.000000,0.703000,0.004020,0.075000,-10.121000,0.043800,140.035000,0.965000,33.000000,9.0,15147.0
8455,1,40,0.124250,0.469050,272470.300000,0.758650,0.608940,0.123650,-7.041000,0.065040,129.251950,0.397725,40.800000,4.0,719161.0
5744,1,18,0.159020,0.725000,216509.777778,0.701222,0.018432,0.123522,-9.669444,0.049800,127.438333,0.870000,48.444444,7.0,2109889.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,1,2,0.035100,0.549000,228904.000000,0.656000,0.000001,0.129000,-6.515000,0.043800,180.043000,0.594000,62.000000,3.0,1306422.0
6056,1,21,0.355819,0.561143,360548.333333,0.546714,0.000009,0.472410,-9.256429,0.048143,109.815714,0.326867,40.714286,10.0,129736.0
2682,1,4,0.145500,0.785000,167627.000000,0.716000,0.000023,0.110450,-5.511500,0.128450,130.499000,0.839500,31.500000,7.0,308642.0
6443,1,14,0.314571,0.478286,154222.857143,0.426000,0.000002,0.158357,-10.463429,0.031329,121.012714,0.535143,47.285714,0.0,739696.0


#Model Analysis

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from sklearn.model_selection import validation_curve
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns; sns.set_style('white')  
import warnings

warnings.filterwarnings('ignore')

# Initializing Classifiers
clf1 = LogisticRegression(multi_class='multinomial',
                          solver='newton-cg',
                          random_state=23225)
clf2 = KNeighborsClassifier(algorithm='ball_tree',
                            leaf_size=50)
clf3 = RandomForestClassifier()



# Building the pipelines
pipe1 = Pipeline([('std', StandardScaler()),
                  ('classifier', clf1)])

pipe2 = Pipeline([('std', StandardScaler()),
                  ('classifier', clf2)])


pipe3 = Pipeline([('classifier', clf3)]) 


#Choosing k between 1 and 1000, 26 values
k_flt = np.logspace(0.1, 3, num = 26)
k_list = []
for num in k_flt:
  k_list.append(int(num))

# Setting up the parameter grids
param_grid1 = [{'classifier__penalty': ['none','l2'],
                'classifier__C': np.power(10., np.arange(-4, 4))}] #Varying C

param_grid2 = [{'classifier__n_neighbors': k_list, #Varying K 
                'classifier__p': [1, 2]}]

param_grid3 = [{'classifier__n_estimators': [10, 100, 1000],
                 'classifier__max_features': [1, 2,4,6,8,12,16,20]}]



# Setting up multiple GridSearchCV objects, 1 for each algorithm
gridcvs = {}

for pgrid, est, name in zip((param_grid1, param_grid2, param_grid3),
                            (pipe1, pipe2, pipe3),
                            ('Logistic', 'KNN', 'RF')):
  
    gcv = GridSearchCV(estimator=est,
                       param_grid=pgrid,
                       scoring='accuracy',
                       n_jobs=1,
                       cv=2, # just 2-fold inner loop, i.e. train/test
                       verbose=0,
                       refit=True)
    gridcvs[name] = gcv

In [15]:
%%time 

cv_scores = {name: [] for name, gs_est in gridcvs.items()}

skfold = StratifiedKFold( n_splits=3, shuffle=True, random_state=1)

# The outer loop for algorithm selection
c = 1
for outer_train_idx, outer_valid_idx in skfold.split(X_train,y_train):
    for name, gs_est in sorted(gridcvs.items()):
        print('outer fold %d/5 | tuning %-8s' % (c, name), end='')

        # The inner loop for hyperparameter tuning
        gs_est.fit(X_train.iloc[outer_train_idx], y_train.iloc[outer_train_idx])
        y_pred = gs_est.predict(X_train.iloc[outer_valid_idx])
        acc = accuracy_score(y_true=y_train.iloc[outer_valid_idx], y_pred=y_pred)
        print(' | inner ACC %.2f%% | outer ACC %.2f%%' %
              (gs_est.best_score_ * 100, acc * 100))
        cv_scores[name].append(acc)

    c += 1




outer fold 1/5 | tuning KNN      | inner ACC 45.91% | outer ACC 48.94%
outer fold 1/5 | tuning Logistic | inner ACC 47.28% | outer ACC 46.76%
outer fold 1/5 | tuning RF       | inner ACC 50.14% | outer ACC 52.05%
outer fold 2/5 | tuning KNN      | inner ACC 46.72% | outer ACC 48.10%
outer fold 2/5 | tuning Logistic | inner ACC 48.13% | outer ACC 47.25%
outer fold 2/5 | tuning RF       | inner ACC 51.20% | outer ACC 51.55%
outer fold 3/5 | tuning KNN      | inner ACC 46.12% | outer ACC 47.18%
outer fold 3/5 | tuning Logistic | inner ACC 45.98% | outer ACC 47.95%
outer fold 3/5 | tuning RF       | inner ACC 50.18% | outer ACC 51.62%
CPU times: user 27min 53s, sys: 30min 50s, total: 58min 43s
Wall time: 19min 43s


In [16]:
# Looking at the results
for name in cv_scores:
    print('%-8s | outer CV acc. %.2f%% +\- %.3f' % (
          name, 100 * np.mean(cv_scores[name]), 100 * np.std(cv_scores[name])))
print()
for name in cv_scores:
    print('{} best parameters'.format(name), gridcvs[name].best_params_)

Logistic | outer CV acc. 47.32% +\- 0.492
KNN      | outer CV acc. 48.07% +\- 0.720
RF       | outer CV acc. 51.74% +\- 0.218

Logistic best parameters {'classifier__C': 10.0, 'classifier__penalty': 'l2'}
KNN best parameters {'classifier__n_neighbors': 18, 'classifier__p': 1}
RF best parameters {'classifier__max_features': 4, 'classifier__n_estimators': 100}


In [ ]:
# Fitting a model to the whole training set
# using the "best" algorithm
best_algorithim = gridcvs['KNN']

best_algo.fit(X_train, y_train)
train_acc = accuracy_score(y_true=y_train, y_pred=best_algorithim.predict(X_train))
test_acc = accuracy_score(y_true=y_test, y_pred=best_algorithim.predict(X_test))

print('Accuracy %.2f%% (average over CV test folds)' %
      (100 * best_algorithim.best_score_))
print('Best Parameters: %s' % gridcvs['KNN'].best_params_)
print('Training Accuracy: %.2f%%' % (100 * train_acc))
print('Test Accuracy: %.2f%%' % (100 * test_acc))

Accuracy 48.39% (average over CV test folds)
Best Parameters: {'classifier__n_neighbors': 23, 'classifier__p': 1}
Training Accuracy: 53.71%
Test Accuracy: 49.82%
